# Multidimensional models


Fitting is not limited to 1D models. The following example demonstrates how to fit a 2D Gaussian peak to a 2D DataArray.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import lmfit
import xarray_lmfit

In [ ]:
# Generate synthetic 2D data
x = np.linspace(-10, 10, 50)
y = np.linspace(-10, 10, 50)
x_arr = xr.DataArray(x, dims=("x",), coords={"x": x})
y_arr = xr.DataArray(y, dims=("y",), coords={"y": y})
z_arr = lmfit.lineshapes.gaussian2d(
    x_arr,
    y_arr,
    amplitude=4.0,
    centerx=0.0,
    centery=0.0,
    sigmax=1.0,
    sigmay=2.0,
).rename("z")

# Add some noise with fixed seed for reproducibility
rng = np.random.default_rng(5)
z_arr = z_arr.copy(data=rng.normal(z_arr, 0.01))

Fitting a 2D model is as simple as providing multiple coordinate names for different independent variables:

In [ ]:
model = lmfit.models.Gaussian2dModel()

result_ds = z_arr.xlm.modelfit(
    ("x", "y"),
    model=model,
    params=model.make_params(
        amplitude=2.0, centerx=0.0, centery=0.0, sigmax=1.0, sigmay=2.0
    ),
)
result_ds

Let's take a look at the best fit and residuals:

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 3), layout="compressed")

z_arr.plot(ax=axs[0], center=False)
axs[0].set_title("Data")

result_ds.modelfit_best_fit.plot(ax=axs[1])
axs[1].set_title("Fit")

(z_arr - result_ds.modelfit_best_fit).plot(ax=axs[2])
axs[2].set_title("Data $-$ Fit")

for ax in axs:
    ax.set_aspect("equal")